In [1]:
import cv2 as cv
import time
import datetime
 
capt = cv.VideoCapture(0)
face_cascade = cv.CascadeClassifier(cv.data.haarcascades + "haarcascade_frontalface_default.xml")
body_cascade = cv.CascadeClassifier(cv.data.haarcascades + "haarcascade_fullbody.xml")

detection = False
detection_stopped_time = None
timer_started = False
SECONDS_TO_RECORD_AFTER_DETECTION = 5

frame_size = (int(capt.get(3)), int(capt.get(4)))
fourcc = cv.VideoWriter_fourcc(*"mp4v")


while True:
    _, frame_vid = capt.read()
    norm_vid = cv.flip(frame_vid, 1)
    grayscl_img = cv.cvtColor(norm_vid, cv.COLOR_BGR2GRAY)
    faces_rect = face_cascade.detectMultiScale(grayscl_img, 1.3, 5)
    bodies_rect = body_cascade.detectMultiScale(grayscl_img, 1.3, 5)

    if len(faces_rect) + len(bodies_rect) > 0:
        if detection:
            timer_started = False
        else:
            detection = True
            current_time = datetime.datetime.now().strftime("%d-%m-%Y-%H-%M-%S")
            out_stream = cv.VideoWriter(f"{current_time}.mp4", fourcc, 20.0, frame_size)
            print("Start Recording!")
    elif detection:
        if timer_started:
            if time.time() - detection_stopped_time >= SECONDS_TO_RECORD_AFTER_DETECTION:
                detection = False
                timer_started = False
                out_stream.release()
                print("Stop Recording!")
        else:
            timer_started = True
            detection_stopped_time = time.time()

    # for (x, y, width, height) in faces_rect:
    #     cv.rectangle(norm_vid, (x, y), (x + width, y + height), (255, 0, 0), 3)

    if detection:
        out_stream.write(norm_vid)
        
    cv.imshow("Camera", norm_vid)
    if cv.waitKey(1) == ord("m"):
        break
out_stream.release()
capt.release()
cv.destroyAllWindows()



Start Recording!
Stop Recording!
Start Recording!
